Following Script:
* Crawls the given folder 
* Extract cluster information from each file
* Picks the revelent clusters
* executes FSL's queryatlas to find the details of the region

In [1]:
import glob
import os
from os.path import join as opj
import re
import pandas as pd
import numpy  as np
from utils import atlasUtility as au
from utils import brainnetomeUtility as bu
from tqdm import tqdm

In [2]:
thresh = 0
folder_path = '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected'

# folder_path = 'clustersHypoConnected'



folder = folder_path.split('/')[-1]


name_csv = folder + '_' +  str(thresh) + '.csv'

lst = glob.glob(opj(folder_path,'*.txt'))
lst.sort()

LIMIT_ROIS = 246

if LIMIT_ROIS:
    lst = lst[0:246]



In [110]:
lst

['../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPosThrRoi0000_cluster.txt',
 '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPosThrRoi0001_cluster.txt',
 '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPosThrRoi0002_cluster.txt',
 '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPosThrRoi0003_cluster.txt',
 '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPosThrRoi0004_cluster.txt',
 '../OHBM2018FinalBrainMaps/fdrRes/motionRegress1global0smoothing1filt1/motionRegress1global0smoothing1filt1/output/clustersHyperConnected/tValPo

In [3]:
atlas_path = 'brainnetomeAtlas/BNA-maxprob-thr0-1mm.nii.gz'
# atlasRegionsDescrpPath = '/home/varun/Projects/fmri/Autism-Connectome-Analysis-brain_connectivity/atlas/BNA_subregions.xlsx'
atlasRegionsDescrpPath = 'brainnetomeAtlas/BNA_subregions_machineReadable.xlsx'
q = bu.queryBrainnetomeROI(atlas_path, atlasRegionsDescrpPath)


atlasPaths1  = ['hoAtlas/HarvardOxford-sub-maxprob-thr0-1mm.nii.gz','hoAtlas/HarvardOxford-cort-maxprob-thr0-1mm.nii.gz',
'cerebellumAtlas/Cerebellum-MNIflirt-maxprob-thr0-1mm.nii.gz']
atlasLabelsPaths1 = ['hoAtlas/HarvardOxford-Subcortical.xml','hoAtlas/HarvardOxford-Cortical.xml', \
'cerebellumAtlas/Cerebellum_MNIflirt.xml']

q1 = au.queryAtlas(atlasPaths1,atlasLabelsPaths1)


# In[71]:
# q1.getAtlasRegions([-6,62,-2])
q1.getAtlasRegions([47,-60, 4])
# q1.getAtlasRegions([33, -6, -6])

# In[72]:

# atlasPath2 = ['juelichAtlas/Juelich-maxprob-thr25-1mm.nii.gz']
atlasPath2 = ['juelichAtlas/Juelich-maxprob-thr0-1mm.nii.gz']
atlasLabelsPath2 = ['juelichAtlas/Juelich.xml']
q2 = au.queryAtlas(atlasPath2,atlasLabelsPath2)

# In[73]:
# q1.getAtlasRegions([33, -6, -6])
q1.getAtlasRegions([47,-60, 4])


(13, 'Middle Temporal Gyrus, temporooccipital part', 1)

In [12]:
q.getAtlasRegions([-33,-45,-27])

(0, None, None, None)

In [112]:
q.queryDict(230)[0]

array(['Subcortical Nuclei', 'BG, Basal Ganglia',
       'BG_dlPu, dorsolateral putamen', '29, -3, 1 '], dtype=object)

In [30]:
q.getAtlasRegions([6,-48,3])

(182, 'Limbic Lobe', 'CG, Cingulate Gyrus', 'CG_A23v, ventral area 23')

In [113]:
x1,y1,z1 = map(int,'65, -29, -13'.split(','))

In [114]:
x1,y1,z1

(65, -29, -13)

In [115]:
table = pd.DataFrame()
for path in lst:
    z = re.match(".*Roi(\d{4})_.",path)
    roi = z.group(1)

    roi = int(roi)+1
    print('ROI: ',roi)
    seed_lobe, seed_gyrus, seed_name, seedCoordMNI = q.queryDict(roi)[0]
#     print(seedCoordMNI)
    [sx,sy,sz] = map(int,seedCoordMNI.split(','))
    
    _,seedNameHO,_ = q1.getAtlasRegions([sx,sy,sz])
    
    seed_hemisphere = 'R' if roi%2.0 == 0 else 'L' # Left Hemi ROI have odd labels
#     print('ROI Number ',roi)
    df = pd.read_table(path)

    df_selected = df.loc[(df['Voxels'] > thresh)]
    df_peak_tval_coordinates = df_selected.as_matrix(['Voxels','MAX','MAX X (mm)' ,'MAX Y (mm)','MAX Z (mm)'])
    
    if len(df_peak_tval_coordinates) > 0:
        print(df_peak_tval_coordinates)
        
        for row in df_peak_tval_coordinates:
            x,y,z = map(int,[row[2],row[3],row[4]])
#             print(row[2],row[3],row[4])
            print(x,y,z)
            coord = str(x)+ ','+ str(y) + ',' + str(z)
            
            target_roi, target_lobe, target_gyrus, region = q.getAtlasRegions([x,y,z])
            
            
                
            _,targetNameHO, atlas_index = q1.getAtlasRegions([x,y,z])

#             TODO
#             if target_roi == 0:
#                 if atlas_index == 2: # cerebellum
#                     target_lobe = 'Cerebellum'
#                 else:
#                     target_lobe = targetNameHO
#                 target_gyrus  = targetNameHO
#                 region = targetNameHO
                
                        
            
            
            target_hemishere = 'R' if x > 0 else 'L'
            if x > 0:
                target_hemishere = 'R'
            if x < 0:
                target_hemishere = 'L'
            else:
                target_hemishere = 'C'
            
#             target_hemishere = 'R' if target_roi%2.0 == 0 else 'L'
            
            d = {'SeedROI':[roi], 'SeedNameBN':[seed_name], 'SeedLobe': [seed_lobe], 'SeedGyrus': [seed_gyrus],'SeedNameHO':[seedNameHO], 'Voxels':[row[0]], 'Max_tvalue':[row[1]], 'PeakMNI(X,Y,Z)':[coord], 'TargetROI':[target_roi], 'TargetLobe' : [target_lobe], 'TargetGyrus': [target_gyrus], 'TargetRegion': [region], 'TargetNameHO' : [targetNameHO], 'TargetHemisphere': [target_hemishere], 'SeedHemisphere':[seed_hemisphere]}
#             var = pd.DataFrame([roi, row[0], row[1], coord, str(out1[0]).split('>')[3], \
#                   str(out2[0]).split('>')[3], \
#                   str(out3[0]).split('>')[3]], columns=['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas'])  
            var = pd.DataFrame(d)
            print("var:",var)
            
#             var.columns = ['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas']
            table = table.append(var)[['SeedROI', 'SeedHemisphere','SeedLobe','SeedGyrus','SeedNameBN', 'SeedNameHO','TargetROI','TargetHemisphere','TargetLobe','TargetGyrus', 'TargetRegion', 'TargetNameHO', 'Voxels', 'Max_tvalue', 'PeakMNI(X,Y,Z)']]
            
#             [var.columns.tolist()]
            
table.to_csv(name_csv)

ROI:  1
ROI:  2
ROI:  3
ROI:  4
ROI:  5
ROI:  6
[[  2.     3.74 -36.     0.    15.  ]]
-36 0 15
var:    Max_tvalue PeakMNI(X,Y,Z)                      SeedGyrus SeedHemisphere  \
0        3.74       -36,0,15  SFG, Superior Frontal Gyrus                R   

        SeedLobe               SeedNameBN    SeedNameHO  SeedROI  \
0  Frontal Lobe   SFG_A9l, lateral area 9  Frontal Pole        6   

          TargetGyrus TargetHemisphere    TargetLobe  \
0  INS, Insular Gyrus                L  Insular Lobe   

               TargetNameHO  TargetROI                        TargetRegion  \
0  Central Opercular Cortex        173  INS_dId, dorsal dysgranular insula   

   Voxels  
0     2.0  
ROI:  7
ROI:  8
ROI:  9
ROI:  10
ROI:  11
[[  3.     3.59 -39.    -3.    18.  ]]
-39 -3 18
var:    Max_tvalue PeakMNI(X,Y,Z)                      SeedGyrus SeedHemisphere  \
0        3.59      -39,-3,18  SFG, Superior Frontal Gyrus                L   

        SeedLobe             SeedNameBN           SeedName

[[  6.     3.44 -48.    48.     6.  ]
 [  1.     3.38 -15.   -45.     3.  ]
 [  1.     3.14 -42.    36.    18.  ]]
-48 48 6
var:    Max_tvalue PeakMNI(X,Y,Z)                   SeedGyrus SeedHemisphere  \
0        3.44       -48,48,6  MFG, Middle Frontal Gyrus               L   

        SeedLobe        SeedNameBN    SeedNameHO  SeedROI  \
0  Frontal Lobe   MFG_A46, area 46  Frontal Pole       19   

                  TargetGyrus TargetHemisphere     TargetLobe  TargetNameHO  \
0  MFG, Middle Frontal Gyrus                 L  Frontal Lobe   Frontal Pole   

   TargetROI                    TargetRegion  Voxels  
0         21  MFG_A9/46v, ventral area 9/46      6.0  
-15 -45 3
var:    Max_tvalue PeakMNI(X,Y,Z)                   SeedGyrus SeedHemisphere  \
0        3.38      -15,-45,3  MFG, Middle Frontal Gyrus               L   

        SeedLobe        SeedNameBN    SeedNameHO  SeedROI TargetGyrus  \
0  Frontal Lobe   MFG_A46, area 46  Frontal Pole       19        None   

  TargetHemisph

var:    Max_tvalue PeakMNI(X,Y,Z)           SeedGyrus SeedHemisphere  \
0        2.98      -36,-3,18  OrG, Orbital Gyrus              R   

        SeedLobe                SeedNameBN    SeedNameHO  SeedROI  \
0  Frontal Lobe   OrG_A11m, medial area 11  Frontal Pole       48   

          TargetGyrus TargetHemisphere    TargetLobe  \
0  INS, Insular Gyrus                L  Insular Lobe   

               TargetNameHO  TargetROI                        TargetRegion  \
0  Central Opercular Cortex        173  INS_dId, dorsal dysgranular insula   

   Voxels  
0     5.0  
21 33 -9
var:    Max_tvalue PeakMNI(X,Y,Z)           SeedGyrus SeedHemisphere  \
0        2.89       21,33,-9  OrG, Orbital Gyrus              R   

        SeedLobe                SeedNameBN    SeedNameHO  SeedROI TargetGyrus  \
0  Frontal Lobe   OrG_A11m, medial area 11  Frontal Pole       48        None   

  TargetHemisphere TargetLobe  TargetNameHO  TargetROI TargetRegion  Voxels  
0                R       None  Fronta

var:    Max_tvalue PeakMNI(X,Y,Z)                     SeedGyrus SeedHemisphere  \
0        2.86     15,-24,-21  STG, Superior Temporal Gyrus              L   

        SeedLobe                SeedNameBN     SeedNameHO  SeedROI  \
0  Temporal Lobe  STG_A38m, medial area 38  Temporal Pole       69   

  TargetGyrus TargetHemisphere TargetLobe TargetNameHO  TargetROI  \
0        None                R       None   Brain-Stem          0   

  TargetRegion  Voxels  
0         None     1.0  
-12 -27 -18
var:    Max_tvalue PeakMNI(X,Y,Z)                     SeedGyrus SeedHemisphere  \
0        3.08    -12,-27,-18  STG, Superior Temporal Gyrus              L   

        SeedLobe                SeedNameBN     SeedNameHO  SeedROI  \
0  Temporal Lobe  STG_A38m, medial area 38  Temporal Pole       69   

  TargetGyrus TargetHemisphere TargetLobe TargetNameHO  TargetROI  \
0        None                R       None   Brain-Stem          0   

  TargetRegion  Voxels  
0         None     1.0  
-36 12 1

var:    Max_tvalue PeakMNI(X,Y,Z)                   SeedGyrus SeedHemisphere  \
0         3.1      -18,21,12  MTG, Middle Temporal Gyrus              L   

        SeedLobe                 SeedNameBN  \
0  Temporal Lobe  MTG_A21r, rostral area 21   

                                 SeedNameHO  SeedROI        TargetGyrus  \
0  Middle Temporal Gyrus, anterior division       83  BG, Basal Ganglia   

  TargetHemisphere          TargetLobe TargetNameHO  TargetROI  \
0                L  Subcortical Nuclei         None        227   

             TargetRegion  Voxels  
0  BG_dCa, dorsal caudate     2.0  
-36 -6 -30
var:    Max_tvalue PeakMNI(X,Y,Z)                   SeedGyrus SeedHemisphere  \
0         3.0     -36,-6,-30  MTG, Middle Temporal Gyrus              L   

        SeedLobe                 SeedNameBN  \
0  Temporal Lobe  MTG_A21r, rostral area 21   

                                 SeedNameHO  SeedROI          TargetGyrus  \
0  Middle Temporal Gyrus, anterior division       83  

var:    Max_tvalue PeakMNI(X,Y,Z)                     SeedGyrus SeedHemisphere  \
0        3.05    -18,-27,-45  ITG, Inferior Temporal Gyrus              R   

        SeedLobe                                SeedNameBN  \
0  Temporal Lobe  ITG_A37elv, extreme lateroventral area37   

                                       SeedNameHO  SeedROI TargetGyrus  \
0  Inferior Temporal Gyrus, temporooccipital part       92        None   

  TargetHemisphere TargetLobe TargetNameHO  TargetROI TargetRegion  Voxels  
0                R       None   Brain-Stem          0         None     4.0  
12 -3 -6
var:    Max_tvalue PeakMNI(X,Y,Z)                     SeedGyrus SeedHemisphere  \
0        2.51       12,-3,-6  ITG, Inferior Temporal Gyrus              R   

        SeedLobe                                SeedNameBN  \
0  Temporal Lobe  ITG_A37elv, extreme lateroventral area37   

                                       SeedNameHO  SeedROI        TargetGyrus  \
0  Inferior Temporal Gyrus, temporooc

var:    Max_tvalue PeakMNI(X,Y,Z)            SeedGyrus SeedHemisphere  \
0        3.16      -33,15,63  FuG, Fusiform Gyrus              R   

        SeedLobe                        SeedNameBN  \
0  Temporal Lobe  FuG_A20rv, rostroventral area 20   

                                     SeedNameHO  SeedROI TargetGyrus  \
0  Temporal Fusiform Cortex, posterior division      104        None   

  TargetHemisphere TargetLobe          TargetNameHO  TargetROI TargetRegion  \
0                R       None  Middle Frontal Gyrus          0         None   

   Voxels  
0     1.0  
ROI:  105
ROI:  106
ROI:  107
[[  1.     2.66 -30.   -42.     0.  ]]
-30 -42 0
var:    Max_tvalue PeakMNI(X,Y,Z)            SeedGyrus SeedHemisphere  \
0        2.66      -30,-42,0  FuG, Fusiform Gyrus              L   

        SeedLobe                       SeedNameBN  \
0  Temporal Lobe  FuG_A37lv, lateroventral area37   

                           SeedNameHO  SeedROI        TargetGyrus  \
0  Temporal Occipital Fu

[[  1.     3.25 -12.   -30.   -18.  ]]
-12 -30 -18
var:    Max_tvalue PeakMNI(X,Y,Z)                      SeedGyrus SeedHemisphere  \
0        3.25    -12,-30,-18  SPL, Superior Parietal Lobule              L   

        SeedLobe                            SeedNameBN  \
0  Parietal Lobe  SPL_A7ip, intraparietal area 7(hIP3)   

                                    SeedNameHO  SeedROI TargetGyrus  \
0  Lateral Occipital Cortex, superior division      133        None   

  TargetHemisphere TargetLobe TargetNameHO  TargetROI TargetRegion  Voxels  
0                R       None   Brain-Stem          0         None     1.0  
ROI:  134
ROI:  135
ROI:  136
ROI:  137
ROI:  138
ROI:  139
ROI:  140
[[  1.     3.16 -27.    15.    60.  ]]
-27 15 60
var:    Max_tvalue PeakMNI(X,Y,Z)                      SeedGyrus SeedHemisphere  \
0        3.16      -27,15,60  IPL, Inferior Parietal Lobule              R   

        SeedLobe                            SeedNameBN  \
0  Parietal Lobe  IPL_A40rd, rostr

var:    Max_tvalue PeakMNI(X,Y,Z)            SeedGyrus SeedHemisphere     SeedLobe  \
0        3.06       36,12,12  CG, Cingulate Gyrus              R  Limbic Lobe   

                 SeedNameBN                           SeedNameHO  SeedROI  \
0  CG_A23v, ventral area 23  Cingulate Gyrus, posterior division      182   

          TargetGyrus TargetHemisphere    TargetLobe  \
0  INS, Insular Gyrus                R  Insular Lobe   

               TargetNameHO  TargetROI                        TargetRegion  \
0  Frontal Operculum Cortex        174  INS_dId, dorsal dysgranular insula   

   Voxels  
0     2.0  
ROI:  183
ROI:  184
ROI:  185
[[  4.     3.86 -36.    -3.    18.  ]
 [  1.     3.58 -15.   -12.    45.  ]]
-36 -3 18
var:    Max_tvalue PeakMNI(X,Y,Z)            SeedGyrus SeedHemisphere     SeedLobe  \
0        3.86      -36,-3,18  CG, Cingulate Gyrus              L  Limbic Lobe   

                SeedNameBN                           SeedNameHO  SeedROI  \
0  CG_A23c, caudal are

var:    Max_tvalue PeakMNI(X,Y,Z)          SeedGyrus SeedHemisphere  \
0        3.25       -63,6,21  BG, Basal Ganglia              R   

             SeedLobe               SeedNameBN SeedNameHO  SeedROI  \
0  Subcortical Nuclei  BG_vCa, ventral caudate       None      220   

             TargetGyrus TargetHemisphere     TargetLobe      TargetNameHO  \
0  PrG, Precentral Gyrus                L  Frontal Lobe   Precentral Gyrus   

   TargetROI                            TargetRegion  Voxels  
0         53  PrG_A4hf, area 4(head and face region)     2.0  
-57 0 -33
var:    Max_tvalue PeakMNI(X,Y,Z)          SeedGyrus SeedHemisphere  \
0        3.14      -57,0,-33  BG, Basal Ganglia              R   

             SeedLobe               SeedNameBN SeedNameHO  SeedROI  \
0  Subcortical Nuclei  BG_vCa, ventral caudate       None      220   

                  TargetGyrus TargetHemisphere     TargetLobe  \
0  MTG, Middle Temporal Gyrus                L  Temporal Lobe   

                  

In [ ]:
x,y,z = map(int,['1','2','3'])

In [ ]:
l =  '5.' '-57.' '-36.'

In [ ]:
map(int,l)

In [ ]:
from tqdm import tqdm

In [ ]:
table

In [ ]:
table.to_csv(name_csv)